In [ ]:
import time; print(time.strftime("The last update of this file: %Y-%m-%d %H:%M:%S", time.gmtime()))
import sys, time
# Establish connection with the UE4 game
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)

# Checking status of connection between UnrealCV and UE4 game

res = client.request('vget /unrealcv/status')
# The image resolution and port is configured in the config file.
print('status: ',res)

In [ ]:
import numpy as np
# SM_DenkMitEdelstahlReinigerSpray_18
# SM_CalgonitFinishKlarspueler_3
# SM_CalgonitFinishVorratspack_12
# SM_DenkMitSpezialsalz_15

# good location 239.593 4.203 174.993
# good rotation -42.805 -178.574 0.000

In [ ]:
set_camera_location = client.request('vset /camera/0/location 239.593 4.203 174.993')
set_camera_rotation = client.request('vset /camera/0/rotation -42.805 -178.574 0.000')

In [ ]:
actor_array = ['SM_DenkMitEdelstahlReinigerSpray_18','SM_CalgonitFinishKlarspueler_3','SM_CalgonitFinishVorratspack_12']
actor_array = np.array(actor_array)

In [ ]:
for i in actor_array:
    print(i)

In [ ]:
# set floor color (front floor)

set_floor_mask_color= client.request('vset /object/Floor/color 0 255 0')
for j in actor_array:
    set_mask_color= client.request('vset /object/'+str(j)+'/color 255 0 0')

In [ ]:
dirName = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/'
viewmode_1 = 'lit'
viewmode_2 = 'object_mask'
viewmode_3 = 'depth'
image_type = '.png'
depth_type = '.npy'

In [ ]:
res_lit = client.request('vget /camera/0/'+str(viewmode_1)+str(" ")+str(dirName)+'lit_1_1'+str(image_type))
res_mask = client.request('vget /camera/0/'+str(viewmode_2)+str(" ")+str(dirName)+'mask_1_1'+str(image_type))
res_depth = client.request('vget /camera/0/'+str(viewmode_3)+str(" ")+str(dirName)+'depth_1_1'+str(depth_type))

In [ ]:
print(res_lit)
print(res_mask)
print(res_depth)

In [ ]:
from matplotlib import pyplot as plt
depth = np.load(res_depth)
plt.imshow(depth, cmap='gray')
plt.show()

In [ ]:
import numpy as np
import cv2
import os

lit = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/lit_1_1.png'
mask = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/mask_1_1.png'

In [14]:
import numpy as np
import cv2
import os

lit = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/lit_1_1.png'
mask = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/mask_1.png'


img = cv2.imread(mask)
rgb_img = cv2.imread(lit)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

## Gen lower mask (0-5) and upper mask (175-180) of RED
mask1 = cv2.inRange(img_hsv, (0,50,20), (5,255,255))
mask2 = cv2.inRange(img_hsv, (175,50,20), (180,255,255))

## Merge the mask and crop the red regions
mask = cv2.bitwise_or(mask1, mask2 )
croped = cv2.bitwise_and(img, img, mask=mask)
f = open('F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/ROI.txt', 'a')
f = open('F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/ROI.txt', 'r+')
f.truncate(0)

image, contours, hierarchy =  cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
for t in range (0,3,1):
    if len(contours)>0:
        print('length is: ',len(contours))
        cnt = contours[t]
        x,y,w,h = cv2.boundingRect(cnt)
        draw_rec_lit_image=cv2.rectangle(rgb_img,(x,y),(x+w,y+h),(0,255,0),0)
        print('info of ',t,' contour ',x,',',y,',',x+w,',',y+h)
        x=x
        y=y
        x_1=x+w
        y_1=y+h
        
        f.write(str(x)+' ')
        f.write(str(y)+' ')
        f.write(str(x_1)+' ')
        f.write(str(y_1)+' ')
        f.write("\n")
        
        test_rgb = cv2.bitwise_and(draw_rec_lit_image, draw_rec_lit_image, mask=mask)
f.close()
## Display
cv2.imshow("mask", mask)
cv2.imshow("croped", croped)
cv2.imshow('lit_ROI', draw_rec_lit_image)
cv2.imshow("test_rgb", test_rgb)

cv2.imwrite('F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/desired_1.png',draw_rec_lit_image)
cv2.imwrite('F:/unreal_cv_documentation/detect_ROI_for_multi_actor/image_test/desired_2.png',test_rgb)

if cv2.waitKey() == ord('q'): #press q to close the output image window
        cv2.destroyAllWindows()

length is:  3
info of  0  contour  314 , 271 , 339 , 327
length is:  3
info of  1  contour  381 , 240 , 414 , 277
length is:  3
info of  2  contour  437 , 158 , 477 , 197
